# GDPTools and Conus404 processing 
This tutorial demonstrates the use of gdptools, a python package for area-weighted interpolation of *source* gridded datasets, such as conus404, to *target* polygonal geospatial fabrics.  Source datasets can be any gridded dataset that can be opened in XArray.  However it's important to note that gdptools, operations on XArray Datasets or DataArrays with dimentions of (Y,X,Time) generally.  As such climate datasets that have ensemble dimensions will require subsetting by ensemble to obtain the a dataset with the proper dimeions.  The target dataset can be any polygonal dataset that can be read by GeoPandas.  GDPtools also has capabilities of interpolating gridded data to lines as well, but our focus here is interpolating to polygons. 

This is the second in a series of tutorials illustrating how to use gdptools to process conus404 datasets to geospatial fabrics.  In this workflow, conus404 is aggregated to [**NHDPlusV2 snapshot of the Watershed Boundary Dataset HUC12 boundaries.**](https://www.sciencebase.gov/catalog/item/60cb5edfd34e86b938a373f4).  This is a CONUS scale spatial fabric with ~102,000 polygons.  In this tutorial we take advantage of the Hovenweep onprem version of conus404, and use the Jupyter interactive app on Hovenweep process our workflow co-located with the conus404 data, eliminating the overhead of downloading the data.  

We use the HyTest intake catalog to access the `conus404-daily-diagnostic-onprem-hw` version of conus404 on Hovenweep.  In addition we access GFv1.1 via the `'huc12-geoparquet-osn'` entry in the HyTest catalog.  Compared to the `Part 1 - Delaware River Basin` tutorial, the main difference is that to manage file size and memory overhead we process conus404 by year, generating 43 annual netcdf files of the interpolated data.

## Tutorials in this series

- [**Part 1 - Delaware River Basin**](conus404_spatial_aggregation_DRB.ipynb)
- [**Part 2 - Geospatial Fabric v1.1**](conus404_spatial_aggregation_GFv1_1.ipynb)
- [**Part 3 - NHDPlusV2 snapshot of the Watershed Boundary Dataset HUC12 boundaries**](conus404_spatial_aggregation_WBD12.ipynb)

## Part 3 - NHDPlusV2 snapshot of the Watershed Boundary Dataset HUC12 boundaries

In [1]:
# Common python packages
import xarray as xr
import hvplot.xarray
import hvplot.pandas
import hvplot.dask
import intake
import warnings
import intake_xarray
import intake_parquet
import intake_geopandas
import datetime
import holoviews as hv
import numpy as np
import pandas as pd
import geopandas as gpd

# HyRiver packages
from pynhd import NLDI, WaterData
import pygeohydro as gh
# GDPTools packages
from gdptools import AggGen, UserCatData, WeightGen
import os
os.environ["HYRIVER_CACHE_DISABLE"] = "true"

hv.extension("bokeh")
warnings.filterwarnings('ignore')

Here we setup a variable the sets our local context, working on the HPC or working locally on your Desktop.  This just modifies the access point of the conus404 data, using the Hovenweep access for HPC and the OSN pod access for the Desktop.

In [2]:
t_sys = "HPC"  # "HPC" or "Desktop"


### Access data with HyTest intake catalog.  

- Use the `huc12-geoparquet-osn` to read the NHDPlusV2 snapshot of the Watershed Boundary Dataset HUC12 boundaries
- Use the `conus404-daily-diagnostic-onprem-hw` to read conus404

In [3]:
# open the hytest data intake catalog
# hytest_cat = intake.open_catalog("../dataset_catalog/hytest_intake_catalog.yml")
hytest_cat = intake.open_catalog("https://raw.githubusercontent.com/hytest-org/hytest/main/dataset_catalog/hytest_intake_catalog.yml")
list(hytest_cat)

['conus404-catalog',
 'benchmarks-catalog',
 'conus404-drb-eval-tutorial-catalog',
 'nhm-v1.0-daymet-catalog',
 'nhm-v1.1-c404-bc-catalog',
 'nhm-v1.1-gridmet-catalog',
 'trends-and-drivers-catalog',
 'nhm-prms-v1.1-gridmet-format-testing-catalog',
 'nwis-streamflow-usgs-gages-onprem',
 'nwis-streamflow-usgs-gages-osn',
 'nwm21-streamflow-usgs-gages-onprem',
 'nwm21-streamflow-usgs-gages-osn',
 'nwm21-streamflow-cloud',
 'geofabric_v1_1-zip-osn',
 'geofabric_v1_1_POIs_v1_1-osn',
 'geofabric_v1_1_TBtoGFv1_POIs-osn',
 'geofabric_v1_1_nhru_v1_1-osn',
 'geofabric_v1_1_nhru_v1_1_simp-osn',
 'geofabric_v1_1_nsegment_v1_1-osn',
 'gages2_nndar-osn',
 'wbd-zip-osn',
 'huc12-geoparquet-osn',
 'huc12-gpkg-osn',
 'nwm21-scores',
 'lcmap-cloud',
 'rechunking-tutorial-osn',
 'pointsample-tutorial-sites-osn',
 'pointsample-tutorial-output-osn']

We need a column to use as our identifyer.  Printing huc12.columns below to view all the possible columns, we choose the `HUC12` column as our identifyer.

In [11]:
# open the huc12-geoparquet-osn
huc12_access = hytest_cat['huc12-geoparquet-osn']
huc12 = huc12_access.read()
print(huc12.columns)
huc12

Index(['TNMID', 'METASOURCEID', 'SOURCEDATADESC', 'SOURCEORIGINATOR',
       'SOURCEFEATUREID', 'LOADDATE', 'GNIS_ID', 'AREAACRES', 'AREASQKM',
       'STATES', 'HUC12', 'NAME', 'HUTYPE', 'HUMOD', 'TOHUC',
       'NONCONTRIBUTINGAREAACRES', 'NONCONTRIBUTINGAREASQKM', 'GLOBALID',
       'SHAPE_Length', 'SHAPE_Area', 'geometry'],
      dtype='object')


,TNMID,METASOURCEID,SOURCEDATADESC,SOURCEORIGINATOR,SOURCEFEATUREID,LOADDATE,GNIS_ID,AREAACRES,AREASQKM,STATES,...,NAME,HUTYPE,HUMOD,TOHUC,NONCONTRIBUTINGAREAACRES,NONCONTRIBUTINGAREASQKM,GLOBALID,SHAPE_Length,SHAPE_Area,geometry
0,{B1EF0C55-72ED-4FF6-A3BA-97A87C6A6C47},None,None,None,None,2013-01-18 07:07:56+00:00,NaN,12663.63,51.25,AL,...,Pond Creek,S,NM,031401030102,0.0,0.0,{8BA048A6-E29C-11E2-8094-0021280458E6},0.396873,0.004859,"MULTIPOLYGON (((-86.15784 31.42164, -86.15783 ..."
1,{F0D9874D-52BA-4FDC-A5E6-E259B627764D},None,None,None,None,2013-01-18 07:07:56+00:00,NaN,37030.62,149.86,AL,...,Lightwood Knot Creek,S,NM,031401030103,0.0,0.0,{8BA06091-E29C-11E2-8094-0021280458E6},0.845522,0.014214,"MULTIPOLYGON (((-86.18406 31.53503, -86.18406 ..."
2,{2E0CB201-5672-45B5-8CA7-A60070122697},None,None,None,None,2013-01-18 07:07:56+00:00,NaN,26011.73,105.27,AL,...,Poley Creek-Lightwood Knot Creek,S,NM,031401030302,0.0,0.0,{8BA07E87-E29C-11E2-8094-0021280458E6},0.663941,0.009979,"MULTIPOLYGON (((-86.29029 31.27059, -86.29089 ..."
3,{9D39E120-C6DF-401F-AA8F-1748E9423AA0},None,None,None,None,2013-01-18 07:07:56+00:00,NaN,25800.09,104.41,AL,...,Yellow River,S,NM,031401030302,0.0,0.0,{8BA0956B-E29C-11E2-8094-0021280458E6},0.640934,0.009897,"MULTIPOLYGON (((-86.30253 31.45077, -86.30251 ..."
4,{9ED42D2B-5055-43DD-8096-09B30BB23E24},None,None,None,None,2013-01-18 07:07:56+00:00,NaN,34025.86,137.70,AL,...,Bay Branch Creek,S,NM,031401030203,0.0,0.0,{8BA0B374-E29C-11E2-8094-0021280458E6},0.587969,0.013044,"MULTIPOLYGON (((-86.38404 31.37613, -86.38361 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101720,{38A63408-1F8A-4AC6-B1CF-2BA813CDF11E},{3BE315E6-ED72-4D29-BD78-CFF342F864E7},None,None,None,2019-05-23 13:17:48+00:00,NaN,33389.17,135.12,CN,...,Sandstone Creek,S,NM,040101010103,NaN,NaN,{2536D410-0E91-430F-A7D4-3214661F8CC2},0.995612,0.016356,"MULTIPOLYGON (((-90.16056 48.29768, -90.15993 ..."
101721,{2D6DDDD7-7CBA-4A9B-9489-D45A020120FE},{3BE315E6-ED72-4D29-BD78-CFF342F864E7},None,None,None,2019-05-23 13:17:48+00:00,NaN,20741.07,83.94,CN,...,Lomond River,S,NM,041800000200,NaN,NaN,{33AD1138-9796-4A57-9CCF-B0FA721CF613},0.695497,0.010167,"MULTIPOLYGON (((-89.28114 48.31609, -89.28036 ..."
101722,{CDEAA6BC-B456-4700-B407-DDB69951A5E3},{5C0855E2-B935-4166-B615-1A486A2094C5},None,None,None,2019-02-20 10:48:01+00:00,NaN,405202.45,1639.80,"CN,MI",...,Joseph Island-Frontal Lake Huron,F,NM,042400000200,NaN,NaN,{84A12C64-5E64-4DDC-BF44-08895ABC98B8},92.497572,0.187755,"MULTIPOLYGON (((-83.91464 46.2438, -83.91381 4..."
101723,{BDB7FFB5-3F85-417B-B587-A707F5F68C11},{5C0855E2-B935-4166-B615-1A486A2094C5},None,None,None,2019-02-20 10:48:11+00:00,NaN,1048265.15,4242.18,"CN,MI",...,Manitoulin Island,I,NM,042400000200,NaN,NaN,{CB95FD8D-2EF6-4749-8C0C-9B3743214F10},102.788770,0.490702,"MULTIPOLYGON (((-82.79892 45.9842, -82.79889 4..."


### Load the conus404 dataset using the HyTest catalog

- In this case we are running this notebook on Hovenweep.

In [5]:
# open the hytest data intake catalog
hytest_cat = intake.open_catalog("https://raw.githubusercontent.com/hytest-org/hytest/main/dataset_catalog/hytest_intake_catalog.yml")
list(hytest_cat)

['conus404-catalog',
 'benchmarks-catalog',
 'conus404-drb-eval-tutorial-catalog',
 'nhm-v1.0-daymet-catalog',
 'nhm-v1.1-c404-bc-catalog',
 'nhm-v1.1-gridmet-catalog',
 'trends-and-drivers-catalog',
 'nhm-prms-v1.1-gridmet-format-testing-catalog',
 'nwis-streamflow-usgs-gages-onprem',
 'nwis-streamflow-usgs-gages-osn',
 'nwm21-streamflow-usgs-gages-onprem',
 'nwm21-streamflow-usgs-gages-osn',
 'nwm21-streamflow-cloud',
 'geofabric_v1_1-zip-osn',
 'geofabric_v1_1_POIs_v1_1-osn',
 'geofabric_v1_1_TBtoGFv1_POIs-osn',
 'geofabric_v1_1_nhru_v1_1-osn',
 'geofabric_v1_1_nhru_v1_1_simp-osn',
 'geofabric_v1_1_nsegment_v1_1-osn',
 'gages2_nndar-osn',
 'wbd-zip-osn',
 'huc12-geoparquet-osn',
 'huc12-gpkg-osn',
 'nwm21-scores',
 'lcmap-cloud',
 'rechunking-tutorial-osn',
 'pointsample-tutorial-sites-osn',
 'pointsample-tutorial-output-osn']

In [6]:
# open the conus404 sub-catalog
cat = hytest_cat['conus404-catalog']
list(cat)

['conus404-hourly-onprem-hw',
 'conus404-hourly-cloud',
 'conus404-hourly-osn',
 'conus404-daily-diagnostic-onprem-hw',
 'conus404-daily-diagnostic-cloud',
 'conus404-daily-diagnostic-osn',
 'conus404-daily-onprem-hw',
 'conus404-daily-cloud',
 'conus404-daily-osn',
 'conus404-monthly-onprem-hw',
 'conus404-monthly-cloud',
 'conus404-monthly-osn',
 'conus404-hourly-ba-onprem-hw',
 'conus404-hourly-ba-osn',
 'conus404-daily-ba-onprem',
 'conus404-daily-ba-osn',
 'conus404-pgw-hourly-onprem-hw',
 'conus404-pgw-hourly-osn',
 'conus404-pgw-daily-diagnostic-onprem-hw',
 'conus404-pgw-daily-diagnostic-osn']

There are a couple of options for accessing **conus404**:

1. **HPC Setting (`t_sys = HPC`)**:
    - **Assumption**: The notebook is run on the USGS HPC Hovenweep.
    - **Access Method**: Utilizes the on-premises version of the data.
    - **Benefits**:
        - **Workflow Association**: The workflow is directly linked to the data.
        - **Speed**: Eliminates the need to download data, significantly reducing access and processing time.

2. **Desktop Setting (`t_sys = Desktop`)**:
    - **Use Case**: Suitable for workflows that do not require HPC resources or for developing workflows locally before deploying them to the HPC.
    - **Access Method**: Connects to the **conus404** data via the OSN pod.
    - **Benefits**:
        - **Flexibility**: Allows for local development and testing.
        - **Performance**: Provides a fast connection to the data.


In [7]:
## Select the dataset you want to read into your notebook and preview its metadata
if t_sys == "HPC":
    dataset = 'conus404-daily-diagnostic-onprem-hw'
elif t_sys == "Desktop":
    dataset = 'conus404-daily-diagnostic-osn' 
else:
    print("Please set the variable t_sys above to one of 'HPC' or 'Desktop'")        
cat[dataset]

conus404-daily-diagnostic-onprem-hw:
  args:
    consolidated: true
    urlpath: /caldera/hovenweep/projects/usgs/water/impd/hytest/conus404/conus404_daily_xtrm.zarr
  description: 'CONUS404 daily diagnostic output (maximum, minimum, mean, and standard
    deviation) for water vapor (Q2), grid-scale precipitation (RAINNC), skin temperature
    (SKINTEMP), wind speed at 10 meter height (SPDUV10), temperature at 2 meter height
    (T2), and U- and V-component of wind at 10 meters with respect to model grid (U10,
    V10). These files were created wrfxtrm model output files (see ScienceBase data
    release for more details: https://doi.org/10.5066/P9PHPK4F). This dataset is stored
    on USGS on-premise Caldera storage for Hovenweep and is only accessible via the
    USGS Hovenweep supercomputer.'
  driver: intake_xarray.xzarr.ZarrSource
  metadata:
    catalog_dir: https://raw.githubusercontent.com/hytest-org/hytest/main/dataset_catalog/subcatalogs


In [8]:
# read in the dataset and use metpy to parse the crs information on the dataset
print(f"Reading {dataset} metadata...", end='')
ds = cat[dataset].to_dask().metpy.parse_cf()
ds

Reading conus404-daily-diagnostic-onprem-hw metadata...

<xarray.Dataset> Size: 3TB
Dimensions:       (y: 1015, x: 1367, time: 15707)
Coordinates:
    lat           (y, x) float32 6MB dask.array<chunksize=(350, 350), meta=np.ndarray>
    lon           (y, x) float32 6MB dask.array<chunksize=(350, 350), meta=np.ndarray>
  * x             (x) float64 11kB -2.732e+06 -2.728e+06 ... 2.728e+06 2.732e+06
  * y             (y) float64 8kB -2.028e+06 -2.024e+06 ... 2.024e+06 2.028e+06
  * time          (time) datetime64[ns] 126kB 1979-10-01 ... 2022-10-01
Data variables: (12/38)
    LANDMASK      (y, x) float32 6MB dask.array<chunksize=(350, 350), meta=np.ndarray>
    Q2MAX         (time, y, x) float32 87GB dask.array<chunksize=(24, 350, 350), meta=np.ndarray>
    Q2MEAN        (time, y, x) float32 87GB dask.array<chunksize=(24, 350, 350), meta=np.ndarray>
    Q2MIN         (time, y, x) float32 87GB dask.array<chunksize=(24, 350, 350), meta=np.ndarray>
    Q2STD         (time, y, x) float32 87GB dask.array<chunksize=(24, 350, 350), meta=np.ndarray>
    RAINCVMAX     (time, y, x) float32 87GB dask.array<chunksize=(24, 350, 350), meta=np.ndarray>
    ...            ...
    U10MEAN       (time, y, x) float32 87GB dask.array<chunksize=(24, 350, 350), meta=np.ndarray>
    U10STD        (time, y, x) float32 87GB dask.array<chunksize=(24, 350, 350), meta=np.ndarray>
    V10MAX        (time, y, x) float32 87GB dask.array<chunksize=(24, 350, 350), meta=np.ndarray>
    V10MEAN       (time, y, x) float32 87GB dask.array<chunksize=(24, 350, 350), meta=np.ndarray>
    V10STD        (time, y, x) float32 87GB dask.array<chunksize=(24, 350, 350), meta=np.ndarray>
    crs           int64 8B ...
Attributes: (12/80)
    BL_PBL_PHYSICS:                  1
    BOTTOM-TOP_GRID_DIMENSION:       51
    BOTTOM-TOP_PATCH_END_STAG:       51
    BOTTOM-TOP_PATCH_END_UNSTAG:     50
    BOTTOM-TOP_PATCH_START_STAG:     1
    BOTTOM-TOP_PATCH_START_UNSTAG:   1
    ...                              ...
    USE_THETA_M:                     0
    WEST-EAST_GRID_DIMENSION:        1368
    WEST-EAST_PATCH_END_STAG:        1368
    WEST-EAST_PATCH_END_UNSTAG:      1367
    WEST-EAST_PATCH_START_STAG:      1
    WEST-EAST_PATCH_START_UNSTAG:    1

### GDPTools Background

In this section, we utilize three data classes from the `gdptools` package: `UserCatData`, `WeightGen`, and `AggGen`.

* [**UserCatData**](https://gdptools.readthedocs.io/en/develop/user_input_data_classes.html):  
  Serves as a data container for both the source and target datasets, along with their associated metadata. The instantiated object `user_data` is employed by both the `WeightGen` and `AggGen` classes.

* [**WeightGen**](https://gdptools.readthedocs.io/en/develop/weight_gen_classes.html):  
  Responsible for calculating the intersected areas between the source and target datasets. It generates normalized area-weights, which are subsequently used by the `AggGen` class to compute interpolated values between the datasets.

* [**AggGen**](https://gdptools.readthedocs.io/en/develop/agg_gen_classes.html):  
  Facilitates the interpolation of target data to match the source data using the areal weights calculated by `WeightGen`. This process is conducted over the time period specified in the `UserCatData` object.

### Instantiation of the `UserCatData` class.

In [12]:
# Coordinate Reference System (CRS) of the conus404 dataset
source_crs = ds.crs.crs_wkt

# Coordinate names of the conus404 dataset
x_coord = "x"
y_coord = "y"
t_coord = "time"

# Time period of interest for areal interpolation of conus404 to DRB HUC12s
# using the AggGen class below. Note: The dates follow the same format as the
# time values in the conus404 dataset.
sdate = "1979-10-01T00:00:00.000000000"
edate = "2022-10-01T00:00:00.000000000"

# Variables from the conus404 dataset used for areal interpolation
variables = ["T2MIN", "T2MAX", "RAINNCVMEAN"]

# CRS of the DRB HUC12 polygons
target_crs = 5070

# Column name for the unique identifier associated with target polygons.
# This ID is used in both the generated weights file and the areal interpolated output.
target_poly_idx = "HUC12"

# Common equal-area CRS for reprojecting both source and target data.
# This CRS is used for calculating areal weights in the WeightGen class.
weight_gen_crs = 5070

# Instantiate the UserCatData class, which serves as a container for both
# source and target datasets, along with associated metadata. The UserCatData
# object provides methods used by the WeightGen and AggGen classes to subset
# and reproject the data.
user_data = UserCatData(
    ds=ds,  # conus404 read from the intake catalog
    proj_ds=source_crs,
    x_coord=x_coord,
    y_coord=y_coord,
    t_coord=t_coord,
    var=variables,
    f_feature=huc12,  # huc121 read above from the intake catalog
    proj_feature=target_crs,
    id_feature=target_poly_idx,
    period=[sdate, edate],
)


bounds:  <class 'numpy.ndarray'> [-10522526.98200231  -2392305.21681825   3660801.77662221
   5005903.62707823]


### Weight Generation with `WeightGen`

In this section, we utilize the `WeightGen` class from the `gdptools` package to calculate the normalized areal weights necessary for interpolating gridded data (`conus404`) to polygonal boundaries (`CONUS HUC12s`). The areal weights represent the proportion of each grid cell that overlaps with each polygon, facilitating accurate **areal interpolation** of the data. These weights are calculated using the `calculate_weights()` method.

**Weight Calculation Process:**

1. **Subset Source Data**: The source data is subset based on the bounds of the target data, with an additional small buffer to ensure coverage. The buffer size is determined based on [specific criteria or methodology].

2. **Create cell boundary GeoDataFrame**: A GeoDataFrame of the cell boundaries is created for each node in the subsetted source data, enabling spatial operations.

3. **Validate Geometries**: The target file is checked for invalid geometries, which can occur due to various reasons such as topology errors. Invalid geometries are fixed using Shapely's `make_valid()` method to prevent failures during intersection calculations.

4. **Calculate and Normalize Areas**: For each polygon, `gdptools` calculates the area of each intersecting grid cell and normalizes it by the total area of the target polygon. This ensures that the weights for each polygon sum to 1, provided the polygon is entirely covered by the source data.
   
   - **Validation**: A quick check on the weights can be performed by grouping the resulting weights by the `target_poly_idx` and calculating the sum. For all polygons completely covered by the source data, the weights will sum to 1.

**Note:** The `method` parameter in `calculate_weights()` can be set to one of `"serial"`, `"parallel"`, or `"dask"`. Given the scale of the gridded `conus404` data (4 km × 4 km) and the spatial footprint of the `CONUS HUC12s`, using `"parallel"`or `"dask"` in this case is the most efficient method.


### Parallel and Dask Methods

The domain in this workflow is large as defined by the number of polygons, the polygon complexity, and the relatively small scale of the conus404 cell geometries.  We can take advantage of the parallel methods to improve performance in both the weight calculation and the interpolation.  The parallel and dask engines used in the `WeightGen` class operate in a similar manner, utilizing Python's `multiprocessing` module and `dask.bag`, respectively.

Using the `jobs` parameter, users can specify the number of processes to run. The target data is divided into chunks based on the number of processes, and each processor receives a chunked `GeoDataFrame` along with a copy of the subsetted source data. This setup introduces overhead that can affect how efficiently the parallel processing runs.

**Trade-offs in Parallel Processing:**

The use of parallel processing involves balancing the number of processors with the overhead of copying data:

- **Benefits**: Increasing the number of processors can reduce computation time by dividing the workload.
- **Costs**: More processors increase memory usage due to duplicate datasets and add coordination overhead between processes.
- **Optimal Performance**: There is a 'sweet spot' where the number of processors maximizes performance. Beyond this point, additional processors may slow down the operation due to overhead.

The optimal number of processors depends on factors such as data size, available memory, and system architecture. It often requires experimentation to determine the most efficient configuration.

In [13]:
%%time
wght_gen = WeightGen(
    user_data=user_data,
    method="parallel",
    output_file="wghts_huc12_c404daily_p.csv",
    weight_gen_crs=weight_gen_crs,
    jobs=4
)

wdf = wght_gen.calculate_weights()

Using parallel engine
Generating grid-cell polygons finished in 105.32 second(s)
Data preparation finished in 105.3273 seconds
     - validating target polygons
     - fixing 8 invalid polygons.
     - validating source polygons
     - fixing 0 invalid polygons.
Validate polygons finished in 749.5435 seconds
     - reprojecting and validating source polygons
     - checking the source polygons for invalid polygons
     - checking source for empty polygons
     - reprojecting and validating target polygons
     - checking the target polygons for invalid polygons
     - checking target for empty polygons
Reprojecting to: EPSG:5070 and validating polygons finished in 57.22 seconds
Weight gen finished in 860.0364 seconds
CPU times: user 16min 19s, sys: 6.13 s, total: 16min 26s
Wall time: 29min 34s


### Compute the areal weighted spatial interpolation

Because the result will be rather large.  To manage the file size and memory requirements for processing we process by year.  Additionaly, The conus404 data starts and ends on the water year dates, so we chose to process by water year in this case.  The code below generates a list of start_dates, end_dates, and years that we iterate over to process the data by year. 

In [14]:
t_start_series = pd.date_range(pd.to_datetime("1979-10-01"), periods=43, freq="YS-OCT")
t_end_series = pd.date_range(pd.to_datetime("1980-09-30"), periods=43, freq="Y-SEP ")
f_time_series = pd.date_range(pd.to_datetime("1980"), periods=43, freq="Y")

time_start = [t.strftime("%Y-%m-%dT%H:%M:%S.%f") for t in t_start_series]
time_end = [t.strftime("%Y-%m-%dT%H:%M:%S.%f") for t in t_end_series]
file_time = [t.strftime("%Y") for t in f_time_series]
time_start[:4], time_end[:4]

(['1979-10-01T00:00:00.000000',
  '1980-10-01T00:00:00.000000',
  '1981-10-01T00:00:00.000000',
  '1982-10-01T00:00:00.000000'],
 ['1980-09-30T00:00:00.000000',
  '1981-09-30T00:00:00.000000',
  '1982-09-30T00:00:00.000000',
  '1983-09-30T00:00:00.000000'])

### Areal Interpolation with the `AggGen` Class

In this section, we demonstrate the use of the `AggGen` class and its `calculate_agg()` method from the `gdptools` package to perform areal interpolation. We will explore all three `agg_engine` options: `"serial"`, `"parallel"`, and `"dask"`. The following links provide detailed documentation on the available parameter options:

* [**agg_engines**](https://gdptools.readthedocs.io/en/develop/agg_gen_classes.html#gdptools.agg_gen.AGGENGINES)
* [**agg_writers**](https://gdptools.readthedocs.io/en/develop/agg_gen_classes.html#gdptools.agg_gen.AGGWRITERS)
* [**stat_methods**](https://gdptools.readthedocs.io/en/develop/agg_gen_classes.html#gdptools.agg_gen.STATSMETHODS)

When using `AggGen` and the `calculate_agg()` method, it is important to consider the overlap between the source and target data when selecting the `stat_method` parameter value. All statistical methods have a masked variant in addition to the standard method; for example, `"mean"` and `"masked_mean"`. In cases where the source data has partial overlap with a target polygon, the `"mean"` method will return a missing value for the polygon, whereas the `"masked_mean"` method will calculate the statistic based on the available overlapping source cells. These considerations help users determine whether using a masked statistic is desirable or if a missing value would be preferred, allowing for post-processing of missing values (e.g., using nearest-neighbor or other approaches to handle the lack of overlap). In the case here conus404 completely covers the footprint of the DRB HUC12s, as such the `"mean"` method would be sufficient. 

Because we are processing by year, we have to create a new UserCatData object for each year processed.  
 

In [15]:
%%time
for index, _ts in enumerate(time_start):
    sdate = time_start[index]
    edate = time_end[index]
    print(sdate, edate)
    user_data = UserCatData(
        ds=ds,  # conus404 read from the intake catalog
        proj_ds=source_crs,
        x_coord=x_coord,
        y_coord=y_coord,
        t_coord=t_coord,
        var=variables,
        f_feature=huc12,  # GFv1.1 read above from the intake catalog
        proj_feature=target_crs,
        id_feature=target_poly_idx,
        period=[sdate, edate],
    )
    
    agg_gen = AggGen(
        user_data=user_data,
        stat_method="mean",
        agg_engine="parallel",
        agg_writer="netcdf",
        weights='wghts_huc12_c404daily_p.csv',
        out_path='.',
        file_prefix=f"{file_time[index]}_huc12_c404_daily_diagnostic",
        jobs=4
    )
    ngdf, ds_out = agg_gen.calculate_agg()

1979-10-01T00:00:00.000000 1980-09-30T00:00:00.000000
bounds:  <class 'numpy.ndarray'> [-10522526.98200231  -2392305.21681825   3660801.77662221
   5005903.62707823]
Processing: T2MIN
    Data prepped for aggregation in 0.0044 seconds
    Data aggregated in 41.1596 seconds
Processing: T2MAX
    Data prepped for aggregation in 0.0071 seconds
    Data aggregated in 38.3325 seconds
Processing: RAINNCVMEAN
    Data prepped for aggregation in 0.0090 seconds
    Data aggregated in 38.6100 seconds
Saving netcdf file to 1980_huc12_c404_daily_diagnostic.nc
1980-10-01T00:00:00.000000 1981-09-30T00:00:00.000000
bounds:  <class 'numpy.ndarray'> [-10522526.98200231  -2392305.21681825   3660801.77662221
   5005903.62707823]
Processing: T2MIN
    Data prepped for aggregation in 0.0023 seconds
    Data aggregated in 36.7219 seconds
Processing: T2MAX
    Data prepped for aggregation in 0.0062 seconds
    Data aggregated in 37.0660 seconds
Processing: RAINNCVMEAN
    Data prepped for aggregation in 0.00

KeyboardInterrupt: 